##### Resources FB Prophet
- https://sungkim11.medium.com/forecast-sales-using-prophet-with-regressors-b19adf8080ab
- https://medium.com/grabngoinfo/multivariate-time-series-forecasting-with-seasonality-and-holiday-effect-using-prophet-in-python-d5d4150eeb57
- https://medium.com/mlearning-ai/multivariate-time-series-forecasting-using-fbprophet-66147f049e66
- https://www.analyticsvidhya.com/blog/2022/04/an-end-to-end-guide-on-time-series-forecasting-using-fbprophet/
- https://facebook.github.io/prophet/docs/quick_start.html
- https://www.jadsmkbdatalab.nl/forecasting-with-facebook-prophet-models-an-intro/

  ##### Algo comparison
- Neptune.AI: https://neptune.ai/blog/arima-vs-prophet-vs-lstm
- Kaggle: https://www.kaggle.com/code/aaronfloreani/forecasting-btc-arima-xgboost-prophet-lstm
- YouTube: https://www.youtube.com/watch?v=0gXeXtL_KjY

##### Notes
- Difference between prophet & fbprophet: (From v0.6 onwards, Python 2 is no longer  supported. As of v1.0, the package name on PyPI is "prophet"; prior to v1.0 it was "fbprophet".)
- For prediction windows we use cross validation, this is applicable to the trainig data.
- Making predictions on the test-set that has regressors is a 'cheat', as we are predicting data we can already see. (https://sungkim11.medium.com/forecast-sales-using-prophet-with-regressors-b19adf8080ab)

In [16]:

#import util
import logging
import warnings
import itertools
import numpy as np
import pandas as pd
from prophet import Prophet
import plotly.graph_objs as go
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from prophet.serialize import model_to_json, model_from_json
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_plotly, plot_components_plotly, add_changepoints_to_plot


logging.getLogger('prophet').setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('data/pyn_m.csv')
df.head()
#df.describe().transpose()

In [ ]:
#Split Training and testing data automatically by percentage insted of manually by dates.
x=df['ds']
y=df['y']
train_df, test_df = train_test_split(df, test_size=0.2, shuffle=False, random_state=42)
train_df.tail()

Hyperparameter tuning

##### Lasso Regression = L1, Ridge Regression = L2
- Because prophet has many places possible places for rate change, it uses L1 regularization
- By default changepoints are only inferred for the first 80% of the time series
 - If the trend changes are being overfit (too much flexibility) or underfit (not enough flexibility), you can adjust the strength of the sparse prior using the input argument changepoint_prior_scale. By default, this parameter is set to 0.05. Increasing it will make the trend more flexible:

In [ ]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}
#create cutoff dates
cutoffs = pd.to_datetime(['2013-02-15', '2013-03-15', '2013-04-15']) #Manually select dates 30 days apart
# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here

In [ ]:
# Use cross validation to evaluate all parameters
for params in all_params:
    m = Prophet(**params).fit(train_df)  # Fit model with given params
    df_cv = cross_validation(m, cutoffs=cutoffs, horizon='30 days', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

In [ ]:
# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses

#best params
best_params = all_params[np.argmin(rmses)]
print(best_params)

In [ ]:
#Add country holidays model.add_country_holidays(country_name='UK')
model = Prophet(changepoint_prior_scale=0.1, seasonality_prior_scale=10.0) #Use best parameters
model.add_country_holidays(country_name='US')
model.fit(train_df)

In [ ]:
future = model.make_future_dataframe(periods=30, freq='D') #makefuture dataframe 30 days
forecast = model.predict(future)
plot_plotly(model, forecast)

##### Cross validation
- Horizon: the time to be predicted
-  Initial: the initial training period
- Period: the spacing between cut-off dates

In [ ]:
#Here we do cross-validation to assess prediction performance on a horizon of 30 days,
# starting with 730 days of training data in the first cutoff and then making predictions after every  180 days.

df_cv = cross_validation(model, initial='730 days', period='180 days', horizon = '30 days')
df_cv.tail()

In [ ]:
cutoff = df_cv['cutoff'].unique()[0]
df_cv = df_cv[df_cv['cutoff'].values == cutoff]

###### Plot Cross Validation

In [ ]:
fig = plt.figure(facecolor='w', figsize=(10, 6))
ax = fig.add_subplot(111)
ax.plot(model.history['ds'].values, model.history['y'], 'k.')
ax.plot(df_cv['ds'].values, df_cv['yhat'], ls='-', c='#0072B2')
ax.fill_between(df_cv['ds'].values, df_cv['yhat_lower'],
                df_cv['yhat_upper'], color='#0072B2',
                alpha=0.2)
ax.axvline(x=pd.to_datetime(cutoff), c='gray', lw=4, alpha=0.5)
ax.set_ylabel('y')
ax.set_xlabel('ds')
ax.text(x=pd.to_datetime('2009-01-01'),y=12, s='Initial', color='black', #Initial training period 730 days
       fontsize=16, fontweight='bold', alpha=0.8)
ax.text(x=pd.to_datetime('2010-05-07'),y=12, s='Cutoff', color='black',
       fontsize=16, fontweight='bold', alpha=0.8)
ax.axvline(x=pd.to_datetime(cutoff) + pd.Timedelta('30 days'), c='gray', lw=4,
           alpha=0.5, ls='--')
ax.text(x=pd.to_datetime('2010-02-07'),y=6, s='Horizon', color='black',
       fontsize=11, fontweight='bold', alpha=0.9)

##### Perfomance metrics

In [ ]:
perf_df = performance_metrics(df_cv)
perf_df.head()

In [ ]:
#Plot change points
fig = model.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), model, forecast)

In [ ]:
#Checkout holidays included in model training (model.train_holiday_names)
model.train_holiday_names

In [ ]:
fig = model.plot_components(forecast)

##### Save model

In [17]:
with open('serialized_model.json', 'w') as fout:
    fout.write(model_to_json(model)) #save model

##### Load model and perfom inference

In [ ]:
with open('serialized_model.json', 'r') as fin:
    saved_model = model_from_json(fin.read()) #load model 